In [1]:
import os
import re
import sys
#import tensorflow as tf
#import tensorflow.python.platform
#from tensorflow.python.platform import gfile
import numpy as np
import pandas as pd
import sklearn
from sklearn import cross_validation
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC, LinearSVC
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV

C:\Users\abhanand\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
features = pd.read_csv('train_features/train_features.csv')
labels = pd.read_csv('train_features/train_labels.csv')

In [3]:
features = np.asarray(features.iloc[:,1:])
#labels = np.asarray(labels.iloc[:,0])
labels=np.asarray(labels.iloc[:,1])

In [18]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(features, labels, test_size=0.2, random_state=42)

In [19]:
y_train

array(['096.Hooded_Oriole', '154.Red_eyed_Vireo', '129.Song_Sparrow', ...,
       '091.Mockingbird', '114.Black_throated_Sparrow', '061.Heermann_Gull'], dtype=object)

In [20]:
y_train =[y_train[i].split('.')[0] for i in range (len(y_train))]
y_train = [y_train[i].lstrip('0') for i in range (len(y_train))]
y_train

['96',
 '154',
 '129',
 '73',
 '56',
 '131',
 '108',
 '130',
 '177',
 '163',
 '164',
 '199',
 '198',
 '31',
 '46',
 '156',
 '90',
 '121',
 '42',
 '172',
 '38',
 '46',
 '154',
 '74',
 '11',
 '91',
 '66',
 '67',
 '55',
 '195',
 '172',
 '138',
 '197',
 '156',
 '90',
 '74',
 '74',
 '92',
 '35',
 '94',
 '100',
 '82',
 '182',
 '86',
 '63',
 '129',
 '200',
 '26',
 '72',
 '13',
 '27',
 '55',
 '139',
 '42',
 '158',
 '143',
 '74',
 '146',
 '79',
 '21',
 '30',
 '79',
 '164',
 '36',
 '192',
 '144',
 '98',
 '106',
 '87',
 '111',
 '86',
 '96',
 '194',
 '114',
 '191',
 '55',
 '49',
 '134',
 '38',
 '185',
 '45',
 '44',
 '189',
 '70',
 '134',
 '80',
 '104',
 '127',
 '121',
 '134',
 '89',
 '50',
 '139',
 '13',
 '56',
 '59',
 '2',
 '108',
 '180',
 '45',
 '45',
 '68',
 '88',
 '165',
 '158',
 '11',
 '63',
 '137',
 '128',
 '69',
 '95',
 '134',
 '191',
 '79',
 '134',
 '120',
 '185',
 '36',
 '49',
 '10',
 '2',
 '184',
 '123',
 '142',
 '165',
 '62',
 '67',
 '132',
 '134',
 '46',
 '138',
 '42',
 '55',
 '68',
 '

In [24]:
y_train = np.asarray(y_train)

In [25]:
y_train= y_train.astype(object)

In [26]:
y_train

array(['96', '154', '129', ..., '91', '114', '61'], dtype=object)

In [31]:
y_train= y_train.astype(np.int)
y_train

array([ 96, 154, 129, ...,  91, 114,  61])

In [7]:
# lab4-part1.html
def predict(beta, X_test):
    """
    Given new observations in X_test, predict their labels.
    :param beta: Estimated coefficients
    :param X_test: New observations
    :return: predictions: The predicted labels for the observations in X_test
    """
    beta = np.reshape(beta, (65, 10))
    # Note that we only need to compute the numerator of the probability expressions that
    # are earlier in this file, since the denominators are the same for all of them.
    exp_term = np.exp(beta[0, :] + X_test.dot(beta[1:, :]))  # Nxk matrix of exp(beta_{j0} + beta_j^Tx_i) for all i,j
    exp_term = np.hstack((exp_term, np.ones(np.size(exp_term, 0))[:, np.newaxis])) # Add 1's for last term
    predictions = np.argmax(exp_term, axis=1)
    return predictions

In [41]:
# Hw 4 solution 
# Question 1
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV


# Part (a)
def soft_threshold(c, lambduh):
    if c < -lambduh:
        return c+lambduh
    elif c > lambduh:
        return c-lambduh
    else:
        return 0


def coord_descent_solution1d(x, y, lambduh):
    c = 2*x*(y-0)
    a = 2*x**2
    return soft_threshold(c, lambduh)/a


# Part (b)
def coord_descent_solution(j, X, lambduh, beta, a, xy):
    """
    Compute the solution of the coordinate descent problem at one iteration
    :param j: Coordinate to optimize over
    :param X: Matrix of predictor values
    :param lambduh: Regularization parameter
    :param a: Precomputed values a_1,...,a_d
    :param xy: Precomputed values \sum x_{i1}y_i, ..., \sum x_{id}y_i
    :return: beta_j: Updated value of beta_j
    """
    # Method 1:
    # n, d = X.shape
    # c_j = 0
    # for i in range(n):
    #     beta_x = 0
    #     for k in range(0, d):
    #         if k != j:
    #             beta_x += beta[k]*X[i, k]
    #     c_j += 2*X[i, j]*(y[i] - beta_x)
    # Method 2:
    c_j = 2*(xy[j] - np.sum(X[:,j]*(np.inner(X, beta) - X[:, j]*beta[j])))
    # Method 3:
    # beta_minus_j = np.concatenate((beta[0:j], beta[j+1:]))
    # x_minus_j = np.hstack((X[:, 0:j], X[:, j+1:]))
    # c_j = 2*(xy[j] - np.sum(X[:,j]*np.dot(x_minus_j, beta_minus_j[:, np.newaxis]).T))

    return soft_threshold(c_j, lambduh)/a[j]
# Part (d): Objective function
def computeobj(beta, X, y, lambduh):
    return 1/np.size(X, 0)*sum((y-np.dot(X, beta))**2) + lambduh*sum(abs(beta))


# Part (e): Cyclic coordinate descent
def cycliccoorddescent(beta, X, y, lambduh, max_iter):
    d = np.size(X, 1)
    a = 2*np.sum(X**2, axis=0)
    xy = np.dot(X.T, y)
    all_betas = beta
    for i in range(max_iter):
        for j in range(d):
            beta[j] = coord_descent_solution(j, X, lambduh, beta, a, xy)
        all_betas = np.vstack((all_betas, beta))

    return all_betas


# Part (f): Function to randomly select coordinate
def pickcoord(d):
    return np.random.randint(d)


# Part (g): Random coordinate descent
def randcoorddescent(beta, X, y, lambduh, max_iter):
    d = np.size(X, 1)
    a = 2*np.sum(X**2, axis=0)
    xy = np.dot(X.T, y)
    all_betas = beta
    for i in range(max_iter*d):
        j = pickcoord(d)
        beta[j] = coord_descent_solution(j, X, lambduh, beta, a, xy)
        if i % d == 0:
            all_betas = np.vstack((all_betas, beta))

    return all_betas


# Part (h): Get optimal lambda from sklearn, run coord descent, and plot objective values
def sklearn_lasso(X, y):
    lasso = LassoCV(fit_intercept=False).fit(X, y)
    lambda_opt = lasso.alpha_*2  # Note scikit-learn's objective function is different from ours
    beta_star = lasso.coef_
    return beta_star, lambda_opt

In [ ]:
# Initialize things
X = X_train
y = y_train

beta_star, lambda_opt = sklearn_lasso(X, y)
beta_init = np.zeros(np.size(X, 1))
max_iter = 500


C:\Users\abhanand\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\abhanand\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [ ]:
# Optimizing the objective with the 1/n out front will give the same optimal betas as optimizing the
# objective without the 1/n, but with multiplying lambda by n.
betas_cyclic = cycliccoorddescent(beta_init, X, y, lambda_opt*n, max_iter)



In [ ]:
beta_init = np.zeros(np.size(X, 1))
betas_rand = randcoorddescent(beta_init, X, y, lambda_opt*n, max_iter)

In [ ]:
# My algorithm did not stop at the same location as sklearn's, so the estimated betas will be different.
predictions = predict(v_grad[0], X_test)
print("Accuracy: {:.2f}%".format(np.mean(y_test == predictions)*100))